In [ ]:
from tensorflow.keras.models import load_model
import os
import numpy as np
import cv2
class_names = ['Me', 'Not Me']

model = load_model(os.path.join('Models', 'Face_MobNets50.h5'))


In [ ]:
def get_extended_image(img, x, y, w, h, i, k=0.1):
    '''
    Function, that return cropped image from 'img'
    If k=0 returns image, cropped from (x, y) (top left) to (x+w, y+h) (bottom right)
    If k!=0 returns image, cropped from (x-k*w, y-k*h) to (x+k*w, y+(1+k)*h)
    
    Parameters:
        img : The original image
        x : x coordinate of the upper-left corner
        y : y coordinate of the upper-left corner
        w : Width of the desired image
        h : Height of the desired image
        k : The coefficient of expansion of the image

    Returns:
        image (resized mage with extra dimension at axis=0 as a numpy array)
    '''

    # The next code block checks that coordinates will be non-negative
    # (in case if desired image is located in top left corner)
    if x - k*w > 0:
        start_x = int(x - k*w)
    else:
        start_x = x
    if y - k*h > 0:
        start_y = int(y - k*h)
    else:
        start_y = y

    end_x = int(x + (1 + k)*w)
    end_y = int(y + (1 + k)*h)

    face_image = img[start_y:end_y,
                     start_x:end_x]
   
    face_image = cv2.resize(face_image, (224, 224))
    face_image = face_image/255
   
    # shape from (250, 250, 3) to (1, 250, 250, 3)
    face_image = np.expand_dims(face_image, axis=0)
    
    return face_image

In [ ]:
# Colors to draw rectangles in BGR
RED = (0, 0, 255)
GREEN = (0, 255, 0)

In [ ]:
# opencv object that will detect faces for us
face_cascade = cv2.CascadeClassifier(
    cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

In [ ]:
video_capture = cv2.VideoCapture(0)  # webcamera

if not video_capture.isOpened():
    print("Unable to access the camera")
else:
    print("Access to the camera obtained")

print("Streaming started - to quit press ESC")

while True:

    # Capture frame-by-frame
    ret, frame = video_capture.read()


    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    faces = face_cascade.detectMultiScale(
        gray,
        scaleFactor=1.3,
        minNeighbors=5,
        minSize=(100, 100),
        flags=cv2.CASCADE_SCALE_IMAGE
    )

    for (x, y, w, h) in faces:
        # for each face on the image detected obtaining the EXPANDED image

        face_image = get_extended_image(frame, x, y, w, h, i, 0.5)
        # classify face and draw a rectangle around the face
        # green for positive class and red for negative
        
        result = model.predict(face_image)
        
        if float(result) > 0.5:
            key = 1
        else:
            key = 0
        prediction = class_names[np.array(key)]  # predicted class
        confidence = np.array(result[0]).max(axis=0)  # degree of confidence

        if prediction == 'Me':
            color = GREEN
        else:
            color = RED
        # draw a rectangle around the face
        cv2.rectangle(frame, (x, y), (x+w, y+h), color, 2)
        
        cv2.putText(frame, "{:6} - {:.2f}%".format(prediction, confidence*100), (x, y), cv2.FONT_HERSHEY_PLAIN, 2, color, 2)  

    # display the resulting frame
    cv2.imshow("Face detector - to quit press ESC", frame)

    # Exit with ESC
    key = cv2.waitKey(1)
    if key % 256 == 27:  # ESC code
        break
    # break

# when everything done, release the capture
video_capture.release()
cv2.destroyAllWindows()
print("Streaming ended")